# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import pprint as pprint

# Import API key
from api_keys import g_key

#sanity check
#g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [33]:
#read in the csv
file = "output_data/cities.csv"
cities_df = pd.read_csv(file)


#sanity check
#cities_df

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configure gmaps
gmaps.configure(api_key = g_key)

In [31]:
#store the locations
locations = cities_df[["lat", "lng"]].astype(float)

#store the weight
weight = cities_df['Humidity']

In [34]:
%matplotlib notebook
#make the figure with heat layer
fig = gmaps.figure()

#create the heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weight,
                             dissipating=False, max_intensity=95,
                             point_radius=3.0)
#add it to the figure
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#convert the temperature to farenheit
max_temp_data = cities_df['Max Temp'].tolist()

#sanity check
#print(len(max_temp_data))


#convert the temperature data
converted_temp = []

for item in max_temp_data:
    f = (float(item) - 273.15) * 1.8 + 32
    converted_temp.append(f)
    
#add it to the cities_df
cities_df['Max Temp'] = converted_temp

In [7]:
#sanity check
#cities_df

In [8]:
#creating a new df with the specific vacation criteria
criteria_df = cities_df.loc[((cities_df['Max Temp'] < 80) & (cities_df['Max Temp'] > 70))
                           & (cities_df['Wind Speed'] < 10)
                           & (cities_df['Cloudiness'] == 0)]

#sanity check
criteria_df

,Unnamed: 0,city,lat,lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
36,36,atuona,-9.80,-139.03,79.700,72,0,7.93,PF,1595460092
74,74,tukrah,32.53,20.58,75.110,78,0,3.34,LY,1595460257
92,92,tiznit,29.58,-9.50,78.728,42,0,0.88,MA,1595460808
115,115,awjilah,29.11,21.29,76.712,39,0,4.49,LY,1595460813
138,138,vila velha,-20.33,-40.29,75.992,83,0,3.10,BR,1595460256
139,139,lata,40.16,-8.33,71.996,61,0,0.76,PT,1595460248
169,169,herat,34.50,62.00,73.688,19,0,5.19,AF,1595460826
171,171,lingyuan,41.24,119.40,77.594,52,0,2.75,CN,1595460826
177,177,fez,34.04,-5.00,77.000,53,0,3.10,MA,1595460812
221,221,jabinyanah,35.03,10.91,77.000,53,0,1.50,TN,1595460837


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
#lists for the hotel df
name = []
lat_list = []
long_list = []
city_list = []
country_list = []

#url for the request
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#paramters for the search
params = {
    "radius": "5000",
    "keyword": "hotel",
    "key": g_key,
    "location":"31.27,56.81"
}

# use iterrows to iterate through pandas dataframe
for index, row in criteria_df.iterrows():

    #get the lat and long values
    lat = row['lat']
    long = row['lng']
    
    # get restaurant type from df
    location = f'{lat},{long}'
    
    # add keyword to params dict
    params['location'] = location

    response = requests.get(url, params=params).json()

    #sanity check
    #pprint.pprint(response)

    try:
        #save the info for the name, lat, long, city, and country
        current_name = response['results'][0]['name']
        
        name.append(current_name)
        
        current_lat = response['results'][0]['geometry']['location']['lat']
        
        lat_list.append(current_lat)
        
        current_long = response['results'][0]['geometry']['location']['lng']
        
        long_list.append(current_long)
        
        current_city = row['city']
        
        city_list.append(current_city)
        
        current_country = row['Country']
        
        country_list.append(current_country)
        
    #if no hotel is found nearby, move on
    except IndexError:
        pass

#sanity check    
#print(lat_list)

In [10]:
#create the hotel df
hotel_data = {'Hotel Name': name, "City": city_list, "Country":country_list, "Lat":lat_list, "Lng":long_list}

hotel_df = pd.DataFrame(hotel_data)

hotel_df

,Hotel Name,City,Country,Lat,Lng
0,Simplicité Marquises,atuona,PF,-9.807877,-139.030335
1,قاعدة فيقا فيقا,tukrah,LY,32.533817,20.593710
2,Golden Tulip Porto Vitória,vila velha,BR,-20.315673,-40.296026
3,Quinta do Outeiro,lata,PT,40.196185,-8.277937
4,Rujia Hotel,lingyuan,CN,41.255875,119.414300
5,Fes Marriott Hotel Jnan Palace,fez,MA,34.031867,-5.003023
6,Al-Dawood Hotel & Restaurant,dalbandin,PK,28.885069,64.401825
7,Les Sources De Caudalie,gradignan,FR,44.730995,-0.557102
8,Hotel Burdy,bardiyah,LY,31.753159,25.087188
9,Florence Hotel Assiut,asyut,EG,27.187841,31.193165


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [35]:
#save the marker locations
marker_locations = hotel_df[['Lat', 'Lng']]

# Add marker layer ontop of heat map
markers = gmaps.marker_layer(marker_locations,
    info_box_content = hotel_info)
fig.add_layer(markers)
fig


Figure(layout=FigureLayout(height='420px'))